# Simple demonstration of discrete Markov processes

Toni Giorgino (IBF-CNR)

You should run this notebook in an R instance or Google Colab. [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giorginolab/Markov-Tutorial-UniPd-2022/blob/main/R_Markov.ipynb)


## Discrete Markov processes

A (discrete-time) Markov chain (DTMC) is a sequence of random variables, known as a stochastic process, in which the value of the next variable depends only on the value of the current variable, and not any variables in the past.

At any given time the system is in one (and only one) of several *states*. We'll deal
with *homogeneous* MC, so transition between states are governed by constant probabilities which only depend on the current state (memorylessness, or Markovian property).

We copy the matrices encoding two examples seen during the lecture. 

For the sake of illustration, we work with the first ("weather") example.

![weather](img/weather.png)

In [ ]:
# The weather example
weatherP <- matrix( c(0.6,  0.3,  0.1,
                      0.2,  0.3,  0.5,
                      0.4,  0.1,  0.5  ), 
                    nrow = 3, byrow = T)

The one with the "colored blocks" trajectory example was...

In [ ]:
colortrajP <- matrix( c(3/5,  0,  2/5,
                        1/4, 3/4, 0,
                        1/4, 2/4, 1/4  ), 
                      nrow = 3, byrow = T)

Note how the rows sum to unity (conservation of probability): the state must
go somewhere.


## Sampling the chain

The simplest thing we can do is *sample* the chain, i.e.
follow the probabilities to obtain one realization (here for 30 steps).

In [ ]:
# Sample a Markov chain 
sample.markov <- function(P, n, s0=1) {
        N <- nrow(P)             # number of states
        s <- s0                  # set state <- requested initial state
        ret <- s                 # initialize the output sequence
        for (i in 2:n) {         # for the requested number of steps
                p <- P[s, ]      #   take the destination probabilities
                nx <- sample(1:N, size=1, prob=p)
                                 #   extract the next state with those probabilities
                ret <- c(ret, nx)#   record the new state
                s <- nx          #   current state <- next state
        }
        return(ret)
}

In [ ]:
P <- weatherP
# Extract 30 time steps
sample.markov(P, 30)


## Stationary state


Question: if we play the game a very long time, what fraction of time would we spend in each state?

* That is the asymptotic (equilibrium) distribution.
* I.e., the probability of finding the system in a given state.


In [ ]:
# Check the stationary state by extracting 1000 samples
N <- 1000
sN <- sample.markov(P, N)
print(table(sN))


This is just an approximate, compute-intensive 
realization. The power of the Markov formalism lies
in the fact that such quantities can be computed from
the matrix *P* right away. For example,
the stationary distribution is computed
from the first eigenvector (see function `sstate`).

In [ ]:
# Remove imaginary part if small
dropSmallImaginary <- function(x) {
        if (all(Im(z <- zapsmall(x))==0)) as.numeric(z) else x
}

# Extract stationary state as eigenvector of eigenvalue=1
sstate <- function(M) {
        ev<-eigen(t(M))   # compute eigenvectors and eigenvalues
        if(dropSmallImaginary(ev$values[1]) != 1) 
                stop("First EV not 1")
        # normalize the first ev so that its elements sum to 1
        ss <- ev$vectors[,1] / sum(ev$vectors[,1])
        dropSmallImaginary(ss)
}


In [ ]:
ss <- sstate(P)
print(ss)

The exact estimate approximates well the samples.
We represent both graphically.

In [ ]:
plot(table(sN), xlab="State", ylab="Times visited")
# Compare with the theory
points(1:3, N*ss, col=2)


Stationary probabilities can be converted readily
into free energy differences (ΔG), up to an
irrelevant additive constant. Assuming *T* = 298 K,

In [ ]:
kB <- 0.001985875    # kcal/mol/K
dG <- -kB * 298 * log(ss)
dG


in units of kcal/mol. It is convenient to set the lowest-energy 
state (the most stable) to 0 kcal/mol.

In [ ]:
dG-min(dG)


## Recover the transition count matrix from a sequence

Here we do the reverse than before: instead of
using the transition probability matrix to sample a sequence,
we now use a sequence to recover the transition count matrix
(later I'll normalize it to get a transition *probability* matrix).

Technically, I use a trick: I prepare a data frame with one 
column (I) holding the sampled values, and another
(J) holding the same values lagged by  1 time
step. So, I can just count the I,J pairs (with the
`table` function).


In [ ]:
install.packages("zoo")
library(zoo)
dij <- data.frame(i = sN,
                  j = lag(zoo(sN), 1, na.pad=T))
dij <- na.omit(dij)  # The very last transition is not valid
T <- table(dij)
addmargins(T)


Now normalize by rows. Note how close it is to the
original matrix.

In [ ]:

estP <- T/rowSums(T)
estP



## Markovianity

The sequence of states that we sampled is markovian
**by construction**. However, in general (for real-world timeseries)
this is not a given.

To test markovianity, I will counts the transitions I → J but
this time considering separately whether they were preceded by 
K = 1, 2 or 3. In other words, I will count  occurrences of these
sequences: 1 → I → J,  2 → I → J, and 3 → I → J.

Remember that the Markov property reads:

$$ p(s_t | s_{t-1}, s_{t-2}, \dots ) = p(s_t | s_{t-1} ) $$
in our case, J is the destination state $s_t$, I is $s_{t-1}$,
and the preceeding state K is $s_{t-2}$.

As above, I use lags and tabulation.

In [ ]:
dkij <- data.frame(k = sN,
                   i = lag(zoo(sN), 1, na.pad=T),
                   j = lag(zoo(sN), 2, na.pad=T))
dkij <- na.omit(dkij)  # The very last transition is not valid
head(dkij)

...and so on.

Probabilities should be independent from K. They will eventually be, 
if we sample long enough.

In [ ]:
T3<-table(dkij)

# Sequences 1-I-J, i.e. K=1
print(T3[1,,]/rowSums(T3[1,,]))

# Sequences 2-I-J, i.e. K=2
print(T3[2,,]/rowSums(T3[2,,]))

# Sequences 3-I-J, i.e. K=3
print(T3[3,,]/rowSums(T3[3,,]))

## The probabilistic point of view

Markov chains can also be followed stochastically,
propagating the probability density. Again you will realize
that (under suitable conditions usually met), regardless 
of the initial state, the final probability distribution
converges towards the equilibrium one.

Of note, the "memory" of the initial state is lost after
a transient (related to the second eigenvalue).

In [ ]:
# Compute time-dependent probabilities of a Markov chain. Same
# as taking the n-th power of the matrix P. s0 is the initial
# probability distribution over states (must sum to 1). If not
# given, the system is assumed to start from state 1.
probs.markov <- function(P, n, s0 = c(1,rep(0,nrow(P)-1))) {
        N <- nrow(P)
        s <- s0
        ret <- s
        for (i in 2:n) {
                # Propagate probability distribution by multiplying
                # on the right by matrix P for as many steps as requested.
                s <- s %*% P
                ret <- rbind(ret, s)
        }
        return(ret)
}

In [ ]:
pT <- probs.markov(P, 8)
matplot(pT, type="o", 
        xlim = c(1,10),
        xlab="Time (starting from T=1)", 
        ylab="Probability of each state",
        main="Starting from state 1")
points(rep(10,3),ss,col=1:3)

pT <- probs.markov(P, 8, c(0,1,0))
matplot(pT, type="o", 
        xlim = c(1,10),
        xlab="Time (starting from T=1)", 
        ylab="Probability of each state",
        main="Starting from state 3")
points(rep(10,3),ss,col=1:3)

pT <- probs.markov(P, 8, c(0,0,1))
matplot(pT, type="o", 
        xlim = c(1,10),
        xlab="Time (starting from T=1)", 
        ylab="Probability of each state",
        main="Starting from state 3")
points(rep(10,3),ss,col=1:3)